In [1]:
from DatasetHandler.BiwiBrowser import *
biwi = readBIWIDataset(dataFolder = BIWI_SnippedData_folder, labelsTarFile = BIWI_Lebels_file_Local, subjectList = [s for s in range(1, 2)])#

Using TensorFlow backend.


Frames from D:\GoogleDrive\Projects\deep_rl_for_head_pose_est\DeepRL_For_HPE\DatasetHandler\BIWI_Files\BIWI_Samples\hpdb\ have been started to read by 2018-12-19 21:20:16.674532
Subject 01's frames have been started to read 2018-12-19 21:20:16.674532
Subject 01's all frames have been read by 2018-12-19 21:20:16.674532
D:\GoogleDrive\Projects\deep_rl_for_head_pose_est\DeepRL_For_HPE\DatasetHandler\BIWI_Files\db_annotations.tgz has been started to read by 2018-12-19 21:20:16.674532
1 annotations have been read by 2018-12-19 21:20:17.712688


In [2]:
frames, labelsList = [], []
for inputMatrix, labels in biwi:
    frames.append(inputMatrix)
    labelsList.append(labels)

In [3]:
inputMatrix.shape

(10, 480, 640, 3)

In [5]:
inputMatrix, labels = frames[0], labelsList[0]

In [6]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import keras

In [ ]:
# Inspect the model
vgg16.summary()

In [33]:
# Verify things look as expected
full_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________

In [7]:
patch_size = BIWI_Frame_Shape #(48, 64, 3)#

In [8]:
from keras.layers import *
from keras import Model 

In [9]:
keras.backend.clear_session()
def getFinalModel():
    dense_layer_1 = 1#int((patch_size[0] * patch_size[1]) / 1)0010#00000
    dense_layer_2 = 8
    num_classes = 6
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape= patch_size)
    block5_pool = vgg16.get_layer('block5_pool').output
    gate = (int(d) for d in block5_pool.shape[1:])
    #vgg16.summary()
    # This shape has to match the last layer in VGG16 (without top)
    dense_input  = Input(shape=gate) #patch_size) #(15, 20, 512)
    dense_output = Flatten(name='flatten')(dense_input)
    dense_output = Dense(dense_layer_1, activation='relu', name='fc1')(dense_output)
    dense_output = Dense(dense_layer_2, activation='relu', name='fc2')(dense_output)
    dense_output = Dense(num_classes, activation='softmax', name='predictions')(dense_output)

    top_model = Model(inputs=dense_input, outputs=dense_output, name='top_model')
    # Now combine the two models
    full_output = top_model(block5_pool)
    full_model  = Model(inputs=vgg16.input, outputs=full_output)

    for layer in full_model.layers[:15]:
        layer.trainable = False
    full_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return full_model

In [10]:
full_model = getFinalModel()

58892288/58889256 [==============================] - 37s 1us/step


In [11]:
full_model.fit(inputMatrix, labels, batch_size=10, epochs=1, verbose=2)

Epoch 1/1
 - 15s - loss: 129354.6484 - acc: 0.0000e+00


In [12]:
import tensorflow as tf
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)

with tf.Session() as sess:
    print (sess.run(c))

InvalidArgumentError: Cannot assign a device for operation 'MatMul': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[{{node MatMul}} = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](a, b)]]

Caused by op 'MatMul', defined at:
  File "C:\cStorage\Program_Files\Anaconda3-5.2.0\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\cStorage\Program_Files\Anaconda3-5.2.0\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\cStorage\Program_Files\Anaconda3-5.2.0\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\cStorage\Program_Files\Anaconda3-5.2.0\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  